In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
user_item_profile = pd.read_csv(f'medalion_data_store/gold/user_item_profile.csv')
#df_user_user = pd.read_csv('medalion_data_store/silver/transcript_features_profile.csv')

In [4]:
user_item_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               14825 non-null  object 
 1   age                  17000 non-null  int64  
 2   id                   17000 non-null  object 
 3   became_member_on     17000 non-null  object 
 4   income               14825 non-null  float64
 5   bec_memb_year_month  17000 non-null  object 
 6   age_group            17000 non-null  object 
 7   cnt_completed_ofr_G  16994 non-null  float64
 8   cnt_completed_ofr_I  16994 non-null  float64
 9   cnt_completed_ofr_J  16994 non-null  float64
 10  cnt_received_ofr_C   16994 non-null  float64
 11  cnt_received_ofr_G   16994 non-null  float64
 12  cnt_received_ofr_H   16994 non-null  float64
 13  cnt_received_ofr_I   16994 non-null  float64
 14  cnt_received_ofr_J   16994 non-null  float64
 15  cnt_viewed_ofr_C     16994 non-null 

In [4]:
popularity = user_item_profile.groupby('ofr_id_short')['cnt_offer_completed'].sum().reset_index()
popularity = popularity.sort_values(by='cnt_offer_completed', ascending=False)
popularity

def popularity_recommendation(person_id, df0, popularity):
    products_senn = df0[df0['person'] == person_id]['ofr_id_short'].unique()
    recommendation = popularity[~popularity['ofr_id_short'].isin(products_senn)]
    return recommendation['ofr_id_short'].head(5).tolist()

usuario_exemplo = 'user_123'
print(popularity_recommendation('ffff82501cea40309d5fdd7edcca4a07', df0, popularity))

['ofr_F', 'ofr_I', 'ofr_A', 'ofr_B', 'ofr_C']


In [12]:
# Create user-product matrix (filling missing values with 0)
user_item_matrix = df_user_user.loc[:,[x.split('_')[0] == 'cnt' for x in df_user_user.columns]]

# Compute user similarity
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Function to recommend products based on similar users
def recommend_user_user(user_id, user_item_matrix, user_similarity_df):
    if user_id not in user_item_matrix.index:
        return []  # If the user is not in the matrix

    # Identify the most similar user
    similar_users = user_similarity_df[user_id].drop(index=user_id).sort_values(ascending=False)
    similar_user = similar_users.index[0]

    # Find products that the similar user consumed but the current user has not seen
    user_products = set(user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index)
    similar_user_products = set(user_item_matrix.loc[similar_user][user_item_matrix.loc[similar_user] > 0].index)
    recommendations = list(similar_user_products - user_products)

    return recommendations[:5]  # Return top 5 recommendations

# Example recommendation for a specific user
print(recommend_user_user('ffff82501cea40309d5fdd7edcca4a07', user_item_matrix, user_similarity_df))

[]


In [13]:
user_item_matrix

,cnt_ofr_A,cnt_ofr_B,cnt_ofr_C,cnt_ofr_D,cnt_ofr_E,cnt_ofr_F,cnt_ofr_G,cnt_ofr_H,cnt_ofr_I,cnt_ofr_J,cnt_tran,cnt_offer_completed,cnt_offer_received,cnt_offer_viewed,cnt_transaction
0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,2.0,3.0,2.0,8.0,3.0,5.0,4.0,8.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,2.0,2.0,3.0
2,0.0,0.0,2.0,3.0,3.0,3.0,0.0,2.0,0.0,0.0,5.0,3.0,5.0,5.0,5.0
3,1.0,3.0,0.0,0.0,0.0,0.0,5.0,2.0,0.0,0.0,8.0,3.0,5.0,3.0,8.0
4,0.0,0.0,0.0,3.0,0.0,3.0,0.0,2.0,3.0,0.0,12.0,3.0,4.0,4.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,0.0,0.0,0.0,3.0,0.0,0.0,3.0,3.0,0.0,3.0,11.0,3.0,6.0,3.0,11.0
16996,2.0,2.0,0.0,2.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0,3.0,5.0,4.0,6.0
16997,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0,0.0,3.0,2.0,5.0
16998,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,6.0,0.0,12.0,3.0,4.0,4.0,12.0


In [11]:
df_user_user.loc[:,[x.split('_')[0] == 'cnt' for x in df_user_user.columns]]

,cnt_ofr_A,cnt_ofr_B,cnt_ofr_C,cnt_ofr_D,cnt_ofr_E,cnt_ofr_F,cnt_ofr_G,cnt_ofr_H,cnt_ofr_I,cnt_ofr_J,cnt_tran,cnt_offer_completed,cnt_offer_received,cnt_offer_viewed,cnt_transaction
0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,2.0,3.0,2.0,8.0,3.0,5.0,4.0,8.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,2.0,2.0,3.0
2,0.0,0.0,2.0,3.0,3.0,3.0,0.0,2.0,0.0,0.0,5.0,3.0,5.0,5.0,5.0
3,1.0,3.0,0.0,0.0,0.0,0.0,5.0,2.0,0.0,0.0,8.0,3.0,5.0,3.0,8.0
4,0.0,0.0,0.0,3.0,0.0,3.0,0.0,2.0,3.0,0.0,12.0,3.0,4.0,4.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,0.0,0.0,0.0,3.0,0.0,0.0,3.0,3.0,0.0,3.0,11.0,3.0,6.0,3.0,11.0
16996,2.0,2.0,0.0,2.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0,3.0,5.0,4.0,6.0
16997,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0,0.0,3.0,2.0,5.0
16998,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,6.0,0.0,12.0,3.0,4.0,4.0,12.0


In [ ]:
# Selecionar características do produto para recomendação (exemplo: métricas das interações)
features = ['curiosity_vr', 'eagerness_cv', 'overall_cr', 'tran_amoun_mean']
df_produto = df.groupby('ofr_id_short')[features].mean().reset_index()

# Normalizar as características
scaler = StandardScaler()
df_produto_scaled = df_produto.copy()
df_produto_scaled[features] = scaler.fit_transform(df_produto[features])

# Calcular similaridade entre produtos
product_similarity = cosine_similarity(df_produto_scaled[features])
product_similarity_df = pd.DataFrame(product_similarity, index=df_produto_scaled['ofr_id_short'], columns=df_produto_scaled['ofr_id_short'])

# Função para recomendar produtos similares com base nos produtos já consumidos pelo usuário
def recomendar_content_based(usuario_id, df, product_similarity_df):
    produtos_vistos = df[df['person'] == usuario_id]['ofr_id_short'].unique()
    recomendacoes = []

    for produto in produtos_vistos:
        if produto in product_similarity_df.index:
            similares = product_similarity_df[produto].sort_values(ascending=False).index[1:4].tolist()
            recomendacoes.extend(similares)

    # Retornar top 5 recomendações únicas
    return list(set(recomendacoes))[:5]

# Exemplo de recomendação para um usuário específico
print(recomendar_content_based('user_123', df, product_similarity_df))